# Síncrono vs Asíncrono: Notebook de ejemplos prácticos

Este notebook acompaña al documento de lectura y lleva las ideas a la práctica con código mínimo y comentarios detallados. Buscamos entender cómo se comporta el tiempo total y qué ocurre “entre bastidores”.

Cómo usar este notebook:
- Ejecuta las celdas en orden. Cada sección explica primero el concepto y luego muestra código.
- Observa los tiempos impresos; compáralos entre enfoques (síncrono, hilos, asíncrono, procesos).
- Prueba cambiando parámetros (p. ej., número de tareas, `max_workers`, duraciones de `sleep`) y vuelve a ejecutar para ver cómo cambia el resultado.

Guía de navegación:
- Utilidades para medir tiempo y observar memoria
- Sencillo y bloqueante (síncrono)
- Concurrencia con hilos (I/O simulado)
- Asíncrono con asyncio (I/O no bloqueante)
- Paralelismo con procesos (CPU-bound)
- Casos edge y combinaciones
- Notas rápidas sobre memoria y GIL


## Utilidades: medir tiempo y observar memoria

Antes de comparar enfoques, necesitamos medir tiempo y (de forma muy básica) observar memoria:
- `time.perf_counter()`: cronómetro de alta resolución ideal para medir duración de bloques.
- Memoria de proceso: usaremos `resource` (Linux/macOS) o, en Linux, `/proc/self/status` para leer `VmRSS` y `VmSize`.
- Helpers que usaremos a lo largo del notebook:
  - `time_block(label)`: mide y muestra el tiempo de ejecución de un bloque de código.
  - `get_memory_info()`: imprime una vista rápida del uso de memoria del proceso (aproximada; suficiente para fines didácticos).


In [20]:
import os
import time
from contextlib import contextmanager

# Intentamos importar 'resource' para Linux/macOS
try:
    import resource  # type: ignore
    _HAS_RESOURCE = True
except Exception:
    _HAS_RESOURCE = False

@contextmanager
def time_block(label: str = "tarea"):
    """Mide el tiempo transcurrido dentro del bloque.
    Uso:
    with time_block("descargas"):
        ...
    """
    t0 = time.perf_counter()
    try:
        yield
    finally:
        dt = time.perf_counter() - t0
        print(f"[{label}] tiempo: {dt:.3f} s")


def get_memory_info():
    """Devuelve info de memoria del proceso actual en una tupla (rss_kb, texto_libre).
    - En Linux/macOS, intenta usar 'resource' (ru_maxrss)
    - En Linux, como alternativa, lee /proc/self/status
    """
    rss_kb = None
    text = []

    if _HAS_RESOURCE:
        try:
            usage = resource.getrusage(resource.RUSAGE_SELF)
            # Nota: en Linux ru_maxrss suele ser KB; en macOS, bytes/1024
            rss_kb = int(usage.ru_maxrss)
            text.append(f"ru_maxrss: {rss_kb} KB (interprete del SO)")
        except Exception as e:
            text.append(f"resource fallo: {e}")

    # Intento /proc/self/status (Linux)
    status_path = "/proc/self/status"
    if os.path.exists(status_path):
        try:
            with open(status_path, "r") as f:
                for line in f:
                    if line.startswith("VmRSS:"):
                        text.append(line.strip())
                    if line.startswith("VmSize:"):
                        text.append(line.strip())
        except Exception as e:
            text.append(f"/proc/self/status fallo: {e}")

    return rss_kb, " | ".join(text) if text else "sin datos"


# Prueba rápida de utilidades
rss_kb, info = get_memory_info()
print("Mem info inicial:", info)
with time_block("pausa de 0.1s"):
    time.sleep(0.1)



Mem info inicial: ru_maxrss: 134472 KB (interprete del SO) | VmSize:	 1058292 kB | VmRSS:	   68328 kB
[pausa de 0.1s] tiempo: 0.101 s


## 1) Sencillo y bloqueante (síncrono)
En el estilo síncrono, cada paso espera a que termine el anterior. La ventaja es la simplicidad; la desventaja es que el tiempo total crece con cada espera.

Qué observar:
- Con 5 tareas de 0.5 s cada una, el tiempo total será cercano a ~2.5 s.
- Cambia el número de tareas o la duración del `sleep` y compara con las siguientes secciones.


In [21]:
import time

def tarea_bloqueante(i: int) -> str:
    # Simula una operación que tarda (p. ej., E/S)
    time.sleep(0.5)
    return f"ok {i}"

with time_block("sincrono: 5 tareas * 0.5s"):
    resultados = []
    for i in range(5):
        # No se inicia la siguiente hasta que termina la actual
        resultados.append(tarea_bloqueante(i))

print("Resultados:", resultados)


[sincrono: 5 tareas * 0.5s] tiempo: 2.510 s
Resultados: ['ok 0', 'ok 1', 'ok 2', 'ok 3', 'ok 4']


## 2) Concurrencia con hilos (I/O simulado)
Creamos un grupo de hilos (“trabajadores”). Cuando un hilo queda esperando E/S, otro hilo puede ejecutar otra tarea.

Qué observar:
- Con 5 workers y 5 tareas de 0.5 s, el tiempo total se acerca a ~0.5–0.6 s (más sobrecarga), porque varias tareas progresan durante el mismo periodo.
- `as_completed` permite procesar resultados en cuanto estén listos.
Nota: para trabajo de CPU (no E/S), los hilos en CPython no dan paralelismo real por el GIL; veremos procesos más adelante.


In [22]:
from concurrent.futures import ThreadPoolExecutor, as_completed

import time

def tarea_io(i: int) -> str:
    # Simula una espera I/O
    time.sleep(0.5)
    return f"ok {i}"

with time_block("hilos: 5 tareas * 0.5s con 5 workers"):
    with ThreadPoolExecutor(max_workers=5) as ex:
        futuros = [ex.submit(tarea_io, i) for i in range(5)]
        # Procesamos en el orden en que terminen (opcional)
        resultados = [f.result() for f in as_completed(futuros)]

print("Resultados (orden de finalización):", resultados)


[hilos: 5 tareas * 0.5s con 5 workers] tiempo: 0.506 s
Resultados (orden de finalización): ['ok 0', 'ok 2', 'ok 1', 'ok 3', 'ok 4']


### 2.1) Hilos y estado compartido: carrera y Lock
Si varios hilos escriben el mismo dato sin coordinación, aparecen condiciones de carrera: el resultado depende del intercalado de operaciones y puede variar entre ejecuciones.

Qué observar:
- Caso sin lock: el contador final suele ser menor que el esperado (se pierden incrementos).
- Caso con lock: protegemos la sección crítica y el resultado coincide con lo esperado.
Consejo: mantén al mínimo las secciones críticas (dentro del `with lock:`) para reducir contención y evitar bloqueos.


In [23]:
import threading

N = 100_000
contador = 0

# Caso sin lock (puede perder incrementos)
def inc_sin_lock():
    global contador
    for _ in range(N):
        contador += 1  # no atómico

# Caso con lock (correcto)
contador2 = 0
lock = threading.Lock()

def inc_con_lock():
    global contador2
    for _ in range(N):
        with lock:  # sección crítica protegida
            contador2 += 1

# Ejecutamos ambos casos
h1 = threading.Thread(target=inc_sin_lock)
h2 = threading.Thread(target=inc_sin_lock)
with time_block("hilos sin lock"):
    h1.start(); h2.start(); h1.join(); h2.join()
print("Esperado:", 2*N, "Obtenido:", contador)

h3 = threading.Thread(target=inc_con_lock)
h4 = threading.Thread(target=inc_con_lock)
with time_block("hilos con lock"):
    h3.start(); h4.start(); h3.join(); h4.join()
print("Esperado:", 2*N, "Obtenido:", contador2)


[hilos sin lock] tiempo: 0.046 s
Esperado: 200000 Obtenido: 200000
[hilos con lock] tiempo: 0.072 s
Esperado: 200000 Obtenido: 200000


## 3) Asíncrono con asyncio (I/O no bloqueante)

Cuando una tarea necesita esperar (red, disco, temporizadores), puede “ceder el turno” para que otras tareas avancen en el mismo hilo. `asyncio` implementa este estilo cooperativo:

- Event loop: un coordinador que reanuda tareas cuando termina su espera.
- `async`/`await`: puntos donde la tarea se pausa y el event loop puede continuar con otra.
- Ideal para I/O-bound (muchas esperas); no proporciona paralelismo de CPU por sí solo.

Idea visual (línea de tiempo):
```
A: inicia ── await(I/O) ──▶ retoma
B:          avanza mientras A espera ─────────▶
C:                    avanza mientras A espera ──▶
```

Buenas prácticas:
- Evita llamadas bloqueantes dentro de funciones `async` (añaden demoras a todo el conjunto).
- En notebooks, usa top-level `await` (no `asyncio.run`) porque ya hay un event loop activo.
- Para CPU-bound, combina con procesos; para I/O sin bloquear, usa `await` en operaciones no bloqueantes.


In [24]:
import asyncio

async def tarea_io_async(i: int) -> str:
    # Simula E/S no bloqueante: cede el control por 0.5s
    await asyncio.sleep(0.5)
    return f"ok {i}"

async def run_async():
    # Lanzamos varias tareas y esperamos a que todas terminen
    tareas = [asyncio.create_task(tarea_io_async(i)) for i in range(5)]
    return await asyncio.gather(*tareas)

async def _driver():
    # En notebooks, el event loop ya está activo; usamos await en vez de asyncio.run
    with time_block("asyncio: 5 tareas * 0.5s"):
        resultados = await run_async()
        print("Resultados:", resultados)

# Top-level await funciona en Jupyter/IPython
await _driver()


Resultados: ['ok 0', 'ok 1', 'ok 2', 'ok 3', 'ok 4']
[asyncio: 5 tareas * 0.5s] tiempo: 0.503 s


### 3.1) Asíncrono sin concurrencia efectiva
Aquí usamos `await` de forma secuencial: esperamos a que termine la tarea 1 para iniciar la 2, y así sucesivamente. Resultado: el tiempo total crece con el número de tareas.

Cómo hacerlo concurrente: crea todas las tareas primero y espera a todas con `gather` (ver celda anterior de asyncio).

In [25]:
import asyncio

async def run_async_secuencial():
    r = []
    # Ejecutamos una tras otra; el tiempo total será cercano a 5 * 0.3s
    for i in range(5):
        await asyncio.sleep(0.3)
        r.append(i)
    return r

async def _driver_seq():
    # En notebooks, evitamos asyncio.run y usamos top-level await
    with time_block("asyncio secuencial: 5 * 0.3s"):
        print(await run_async_secuencial())

await _driver_seq()


[0, 1, 2, 3, 4]
[asyncio secuencial: 5 * 0.3s] tiempo: 1.506 s


## 4) Paralelismo con procesos (CPU-bound)
Para trabajo intensivo de CPU, varios procesos pueden ejecutar realmente al mismo tiempo en distintos núcleos. A diferencia de los hilos en CPython, cada proceso tiene su propio intérprete y no comparten el mismo GIL.

Qué observar:
- Comparación threads vs processes: en CPU-bound, procesos suelen ser más rápidos (paralelismo real).
- Hay sobrecarga de crear procesos y de enviar/recibir datos (serialización).


In [26]:
from concurrent.futures import ProcessPoolExecutor

def cpu_intensivo(n: int) -> int:
    # Simula trabajo pesado de CPU (sin esperas)
    s = 0
    for i in range(n):
        s += (i * i) % 97
    return s

# Comparamos threads vs processes para CPU-bound
from concurrent.futures import ThreadPoolExecutor

datos = [2_000_00] * 4  # cuatro trabajos similares

with time_block("threads CPU-bound (GIL limita)"):
    with ThreadPoolExecutor() as ex:
        res_threads = list(ex.map(cpu_intensivo, datos))

with time_block("processes CPU-bound (paralelismo real)"):
    with ProcessPoolExecutor() as ex:
        res_processes = list(ex.map(cpu_intensivo, datos))

print("len(res_threads)=", len(res_threads), "len(res_processes)=", len(res_processes))


[threads CPU-bound (GIL limita)] tiempo: 0.129 s


/usr/lib/python3.10/multiprocessing/connection.py:766: RuntimeWarning: coroutine 'run_async' was never awaited
  def __init__(self, conn, dumps, loads):


[processes CPU-bound (paralelismo real)] tiempo: 0.116 s
len(res_threads)= 4 len(res_processes)= 4


### 4.1) Coste de serialización y chunksize (map con procesos)
Los datos pasan entre procesos: suelen serializarse (copiarse) y eso consume tiempo. `map(..., chunksize=k)` agrupa elementos para reducir el número de mensajes.

Qué observar:
- Sin `chunksize`: muchos mensajes pequeños → más overhead.
- Con `chunksize=20`: menos mensajes, mejor rendimiento (según tarea/datos).
Sugerencia: mide con tus propios tamaños para encontrar el punto medio adecuado.


In [27]:
from concurrent.futures import ProcessPoolExecutor

# Función sencilla "CPU-bound" para ejemplificar

def trabajo(x: int) -> int:
    total = 0
    for i in range(50_000):
        total += (x + i) % 97
    return total

valores = list(range(200))

with time_block("processes map sin chunksize"):
    with ProcessPoolExecutor() as ex:
        res1 = list(ex.map(trabajo, valores))

with time_block("processes map con chunksize=20"):
    with ProcessPoolExecutor() as ex:
        res2 = list(ex.map(trabajo, valores, chunksize=20))

print(len(res1), len(res2))


[processes map sin chunksize] tiempo: 0.611 s
[processes map con chunksize=20] tiempo: 0.702 s
200 200


## 5) Casos edge y combinaciones
Aquí probamos situaciones que suelen generar dudas y las relacionamos con la teoría:
- Asíncrono sin concurrencia efectiva.
- Concurrencia sin asíncrono (solo hilos).
- Paralelo “sin estilo asíncrono” (procesos con funciones normales).
- Por qué “paralelo sin concurrencia” no tiene sentido.


### 5.1) Asíncrono sin concurrencia efectiva
Usamos `asyncio` de forma secuencial (esperamos una tarea antes de iniciar la siguiente). No hay progreso en paralelo temporal. Compáralo con la versión que usa `gather`.


In [28]:
# Reutilizamos run_async_secuencial definido antes
async def _driver_edge_seq():
    with time_block("edge: async secuencial (5 * 0.2s)"):
        print(await run_async_secuencial())

await _driver_edge_seq()


[0, 1, 2, 3, 4]
[edge: async secuencial (5 * 0.2s)] tiempo: 1.505 s


### 5.2) Concurrente sin asíncrono (hilos)
Cada tarea es bloqueante, pero como las ejecutamos en hilos, varias progresan durante el mismo periodo. Útil cuando el tiempo se va en esperas de E/S.


In [29]:
from concurrent.futures import ThreadPoolExecutor

import time

def espera(i: int) -> str:
    time.sleep(0.2)
    return f"fin {i}"

with time_block("edge: hilos bloqueantes (5 * 0.2s, 5 workers)"):
    with ThreadPoolExecutor(max_workers=5) as ex:
        print(list(ex.map(espera, range(5))))


['fin 0', 'fin 1', 'fin 2', 'fin 3', 'fin 4']
[edge: hilos bloqueantes (5 * 0.2s, 5 workers)] tiempo: 0.203 s


### 5.3) Paralelo sin “estilo asíncrono” (procesos)
Cada tarea es secuencial (normal), pero se ejecutan al mismo tiempo en distintos núcleos gracias a procesos. Esto sí es paralelismo real de CPU.


In [30]:
from concurrent.futures import ProcessPoolExecutor

def tarea_normal(x: int) -> int:
    total = 0
    for i in range(300_000):
        total += (x + i) % 101
    return total

with time_block("edge: procesos secuenciales en paralelo"):
    with ProcessPoolExecutor() as ex:
        print(list(ex.map(tarea_normal, range(4))))


[14998935, 14998965, 14998995, 14999025]
[edge: procesos secuenciales en paralelo] tiempo: 0.131 s


## 6) Notas rápidas sobre memoria y GIL
- Memoria (muy simplificado): hilos comparten memoria del proceso; procesos usan memoria separada. Compartir datos es fácil con hilos (pero hay que sincronizar); con procesos, hay que comunicar datos (más coste) o usar memoria compartida específica.
- GIL (CPython): un cerrojo del intérprete por proceso. Limita el paralelismo de CPU con hilos, pero no afecta la concurrencia de E/S (al esperar, otros hilos avanzan). Para CPU-bound, prefiere procesos; para mucha E/S, hilos o asyncio funcionan bien.
Sugerencia: mide siempre y elige el enfoque según el cuello de botella (CPU vs E/S).


In [31]:
# Observamos memoria (muy básico)

def crear_lista_grande(n: int):
    # Evitar tamaños enormes; esto es ilustrativo
    return [i for i in range(n)]

rss_kb_before, info_before = get_memory_info()
print("Antes:", info_before)

data = crear_lista_grande(300_000)  # crea cierta presión de memoria
rss_kb_after, info_after = get_memory_info()
print("Después:", info_after)

# Evitar retener memoria innecesaria
del data


Antes: ru_maxrss: 134472 KB (interprete del SO) | VmSize:	 1058320 kB | VmRSS:	   68712 kB
Después: ru_maxrss: 134472 KB (interprete del SO) | VmSize:	 1070680 kB | VmRSS:	   81000 kB
